In [100]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import pydicom
import numpy as np
import os
import glob
from tqdm import tqdm
import warnings
DATA_PATH = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/'
WORKING_DIR = '/kaggle/input/rsna-2024-train/'

import pydicom
import numpy as np
from PIL import Image
from random import randint
from torch.utils.data import DataLoader

import torch

SUBMISSION=True

LUMBAR_LABEL_MAPPING = {"l1_l2":0,"l2_l3":1,"l3_l4":2,"l4_l5":3,"l5_s1":4}
LUMBAR_INT_MAPPING = {value: key for key, value in LUMBAR_LABEL_MAPPING.items()}
SET_TYPE = 'train'
DATASET_SELECTION = 'axial'

In [101]:
MODEL_DIR = "/kaggle/input/rsna-yolo-v1/pytorch/best_20240716/5/new_sagittal_20241110.pt" if DATASET_SELECTION=='sagittal' else "/kaggle/input/rsna-yolo-v1/pytorch/best_20240716/6/best_axial_20241112.pt"
IMAGE_DIR = "/kaggle/working/yolo_set/images/inference/"
process_test = False

In [65]:
!pip install ultralytics # '8.3.23'
import os 
os.environ['WANDB_MODE'] = 'disabled'
os.environ['WANDB_DISABLED'] = 'true'

from ultralytics import YOLO, settings
settings.update({"wandb": False})

In [66]:
set_desc = pd.read_csv(DATA_PATH+SET_TYPE+'_series_descriptions.csv')

In [67]:
set_desc.head()

,study_id,series_id,series_description
0,4003253,702807833,Sagittal T2/STIR
1,4003253,1054713880,Sagittal T1
2,4003253,2448190387,Axial T2
3,4646740,3201256954,Axial T2
4,4646740,3486248476,Sagittal T1


In [68]:
sagittal_t1 = set_desc[set_desc['series_description']=='Sagittal T1']
sagittal = set_desc[set_desc['series_description'].isin(['Sagittal T2/STIR','Sagittal T1'])]
axial = set_desc[set_desc['series_description'].isin(['Axial T2'])]

In [69]:
datasets = {'sagittal':sagittal, 'axial':axial,  'sagittal_t1':sagittal_t1, 'all':set_desc}

### Get study_id_meta

In [70]:
def get_study_id_meta(set_desc, set_type: str = 'train'):
    """
    {
        study_id: {
            folder_path: ...
            series_ids: []
            series_descriptions: []
        }
    }
    """
    
    if set_type not in ("train","test"):
        raise Exception(f"set_type must be equal to 'train' or 'test', set_type = '{set_type}''")
    # First aggregate the descriptions, so we have a list of series descriptions, sort the dataframe by series_id to maintain order mapping
    set_desc_sorted = set_desc.sort_values(['study_id','series_id']).groupby('study_id').agg(list) 
    test_study_ids = set_desc_sorted.index.tolist()

    # All file ids
    test_study_ids_path = [int(i.split("/")[-1]) for i in glob.glob(os.path.join(DATA_PATH,f'{set_type}_images/*'))]

    # Check all study_ids exist if they don't then delete from inference pack
    for ndx, id in enumerate(test_study_ids):
        if id not in test_study_ids_path:
            print(f"study_id {id} not on file")
            del test_study_ids[ndx]

    # Now create meta information, filtering on index
    study_id_meta = {}
    for study_id in test_study_ids:
        folder_path = os.path.join(DATA_PATH,f'{set_type}_images/{study_id}')
        study_id_meta.update({study_id: {
                                    "folder_path": folder_path,
                                    "series_id_files": set_desc_sorted.loc[study_id].series_id,
                                    "series_descriptions": set_desc_sorted.loc[study_id].series_description
                                    }
                             })
        
    return study_id_meta

In [71]:
study_id_meta = get_study_id_meta(datasets[DATASET_SELECTION], SET_TYPE)

In [72]:
len(study_id_meta)

1975

### Datasets (Region of interest derived from coordinates) 

Components:
- CandidateList: list of instances with valid coordinates and their labelling info
- Region of interest calculation: crude bounding box calculation from coordinates
    - cache the data
- Return: Image, Label (severity)

In [73]:
from collections import namedtuple
import functools
import datetime
from torch.utils.data import Dataset
import random
candidate_info_tuple = namedtuple(
    'candidate_info_tuple',
    'row_id, study_id, series_id, instance_number, centre_xy, severity, img_path, width_bbox, height_bbox'
)
SEVERITY_MAPPING = {"Normal/Mild":0,"Moderate":1,"Severe":2}
SEVERITY_WEIGHTING = {0:1.0,1:2.0,2:4.0}

In [74]:
@functools.lru_cache(1, typed=True)
def get_candidate_list(selection='all'):
    
    candidate_dataset = datasets[selection]
    
    candidate_list = []
    
    for ndx, instance in enumerate(candidate_dataset.itertuples(name='Candidate')):
        
        try:
            img_path = f"{DATA_PATH}/{SET_TYPE}_images/{instance.study_id}/{instance.series_id}/"

            images = glob.glob(f"{img_path}/*.dcm")
            
            for inst_path in images:
                
                instance_number = inst_path.split('/')[-1].replace('.dcm', '')

                candidate_list.append(
                    candidate_info_tuple(
                        row_id = None,
                        study_id = instance.study_id, 
                        series_id = instance.series_id, 
                        instance_number = instance_number, 
                        centre_xy = tuple(), 
                        severity = None, 
                        img_path = inst_path,
                        width_bbox=None,
                        height_bbox=None

                    )
                )
            
            if not os.path.exists(img_path):
                print(f"Path not found {img_path}")
                continue

            if ndx % 10000 == 0 and ndx !=0:
                print(f"{datetime.datetime.now()}, {ndx} records processed")
            

        except Exception as e:
            print(instance)
            raise e

    return candidate_list

In [75]:
@functools.lru_cache(1, typed=True)
def get_image(path):
    return pydicom.dcmread(path)

In [76]:
candidate_list = get_candidate_list(DATASET_SELECTION)

In [77]:
candidate_list[1]

candidate_info_tuple(row_id=None, study_id=4003253, series_id=2448190387, instance_number='18', centre_xy=(), severity=None, img_path='/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification//train_images/4003253/2448190387/18.dcm', width_bbox=None, height_bbox=None)

Understanding the image characeristics to see if there is some way we can inform the bounding box width/heights.

In [78]:
import cv2

IMG_RESIZE = (416, 416)

def yolo_preprocess(img: np.array, img_resize: tuple): 
    """
    """

    # Normalize to the range [0, 255] using the actual max value
    image_2d = img.astype(float)
    image_2d = (np.maximum(image_2d, 0) / image_2d.max()) * 255.0
    image_2d = np.uint8(image_2d)
    
    # Resize image
    resized_image = cv2.resize(image_2d, IMG_RESIZE, interpolation=cv2.INTER_LINEAR)
    
    return resized_image


In [79]:
class LumbarYoloInference(Dataset):
    
    def __init__(self,
                val_stride=0,
                val_set=False,
                study_id=None,
                candidate_list=None,
                transform=None,
                sample: int | None = None,
                rand_bbox: bool = False,
                sorted_candidate_list=None, 
                save_to_drive: bool = False):
        
        self.sample = sample
        self.rand_bbox = rand_bbox
        self.save_to_drive = save_to_drive
        
        if candidate_list:
            self.candidate_list = candidate_list.copy()
        else:
            self.candidate_list = get_candidate_list().copy()
        
        # Default to a stratified sampled of candidates by study_id (option to provide custom sorted list on other
        # stratifications).
        if sorted_candidate_list:
            self.sorted_candidate_list = sorted_candidate_list.copy()
        else:
            study_ids = np.unique([x.study_id for x in self.candidate_list])
            self.sorted_candidate_list = np.random.shuffle(study_ids)
        
        if study_id:
            self.candidate_list = [x for x in self.candidate_list if x.study_id == study_id]
        
        if val_set:
            assert val_stride > 0, val_stride
            val_study_ids = self.sorted_candidate_list[::val_stride]
            self.candidate_list = [x for x in self.candidate_list if str(x.study_id) in val_study_ids]
            assert self.candidate_list
        elif val_stride > 0:
            del self.sorted_candidate_list[::val_stride]                        
            self.candidate_list = [x for x in self.candidate_list if str(x.study_id) in self.sorted_candidate_list]            
            assert self.candidate_list
    
        self.transform = transform
    
    def __len__(self):
        if self.sample:
            return min(self.sample, len(self.candidate_list))
        else:
            return len(self.candidate_list)
    
    def __getitem__(self, ndx):
        candidate = self.candidate_list[ndx]

        # Get full image
        image = get_image(candidate.img_path)

        resized_image = yolo_preprocess(image.pixel_array, IMG_RESIZE)

        # Pass in defaults so can be used with DataLoader
        candidate = candidate._replace(
            row_id=candidate.row_id or "",
            severity=candidate.severity or 0,
            centre_xy=candidate.centre_xy or (0, 0),
            width_bbox=candidate.width_bbox or 0,
            height_bbox=candidate.height_bbox or 0
        )
        return resized_image, str(candidate.study_id), str(candidate.series_id), candidate.instance_number


In [80]:
# candidate_list = [i for i in candidate_list if i.study_id == 4003253 and i.series_id == 702807833]


In [81]:
InferenceSet = LumbarYoloInference(candidate_list=candidate_list)

In [82]:
yolo_dirs = ["/kaggle/working/yolo_set/images/inference/"]
for d in yolo_dirs:
    if os.path.exists(d):
        continue
    else:
        os.makedirs(d)

In [83]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np

In [84]:
InferenceLoader = DataLoader(dataset=InferenceSet, batch_size=64, num_workers=4)

In [85]:
InferenceSet[0]

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 2, ..., 3, 1, 1],
        [0, 1, 3, ..., 4, 1, 1],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 '4003253',
 '2448190387',
 '12')

In [86]:
def save_image(image, study_id, series_id, instance_number):    
    image_key = "_".join([str(study_id),str(series_id), str(int(instance_number))]) 
    img_path = f'/kaggle/working/yolo_set/images/inference/{image_key}.png'
    
    # Save image
    cv2.imwrite(img_path,image.numpy())

In [87]:
img_path = "/kaggle/working/yolo_set/images/inference/4003253_702807833_12.png"

In [88]:
for ndx, batch in enumerate(InferenceLoader):
    
    images, study_id, series_id, instance_number = batch

    with ThreadPoolExecutor(max_workers = 8) as executor:
        executor.map(save_image, images, study_id, series_id, instance_number)
            
    if (ndx*64)%1280 == 0:
        print(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}, Processed {ndx*64} images.")
    
    if ndx==len(InferenceLoader):
        print(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}, End")


2024-11-12 10:13:34, Processed 0 images.
2024-11-12 10:13:42, Processed 1280 images.
2024-11-12 10:13:52, Processed 2560 images.
2024-11-12 10:14:01, Processed 3840 images.
2024-11-12 10:14:10, Processed 5120 images.
2024-11-12 10:14:21, Processed 6400 images.
2024-11-12 10:14:29, Processed 7680 images.
2024-11-12 10:14:36, Processed 8960 images.
2024-11-12 10:14:46, Processed 10240 images.
2024-11-12 10:14:54, Processed 11520 images.
2024-11-12 10:15:04, Processed 12800 images.
2024-11-12 10:15:12, Processed 14080 images.
2024-11-12 10:15:21, Processed 15360 images.
2024-11-12 10:15:30, Processed 16640 images.
2024-11-12 10:15:39, Processed 17920 images.
2024-11-12 10:15:47, Processed 19200 images.
2024-11-12 10:15:55, Processed 20480 images.
2024-11-12 10:16:06, Processed 21760 images.
2024-11-12 10:16:16, Processed 23040 images.
2024-11-12 10:16:24, Processed 24320 images.
2024-11-12 10:16:33, Processed 25600 images.
2024-11-12 10:16:42, Processed 26880 images.
2024-11-12 10:16:51, 

## Inference

Inference on YOLO for sagittal.

In [102]:
import glob
# Load a model
model = YOLO(MODEL_DIR)  # pretrained YOLOv8n model

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.to(device)

# Run batched inference on a list of images
files_for_inference = sorted(glob.glob(f"{IMAGE_DIR}*"))
results = model.predict(files_for_inference,task='detect',iou=0.8,stream=True)  # return a list of Results objects

if process_test:
    # Process results list
    for result in results:
        boxes = result.boxes  # Boxes object for bounding box outputs
        masks = result.masks  # Masks object for segmentation masks outputs
        keypoints = result.keypoints  # Keypoints object for pose outputs
        probs = result.probs  # Probs object for classification outputs
        obb = result.obb  # Oriented boxes object for OBB outputs
    #     result.show()  # display to screen
        path = result.path.split("/")[-1].replace(".png","")

In [103]:
import re
def clean_newlines(input_string):
    # Replace multiple newlines with a single newline
    cleaned_string = re.sub(r'\n+', '\n', input_string)
    # If there are newlines at the end, remove them
    cleaned_string = re.sub(r'\n+$', '', cleaned_string)
    return cleaned_string.split("\n")

# Function to extract the numeric values from the filename
def extract_numbers(file_path):
    filename = file_path.split('/')[-1]
    numbers = re.findall(r'\d+', filename)
    return list(map(int, numbers))


In [104]:
import matplotlib.pyplot as plt
import cv2
import os
import glob
import math

def plot_yolo_predictions_and_ground_truth(image_dir, label_dir, model, pattern="*", label_paths=[], results = None, **kwargs):
    """
    Plot YOLO predictions and ground truth labels on images.

    Parameters:
    - image_dir (str): Directory containing the images.
    - label_dir (str): Directory containing the label files.
    - model: YOLO model for making predictions.
    - pattern (str): Pattern to match image files in the directory.
    """
    # Get list of images
    image_paths = sorted(glob.glob(os.path.join(image_dir, pattern)),key=extract_numbers)
    
    # Run batch inference
    results = model(image_paths, **kwargs)

    # Plot image with ground truth and predicted boxes
    num_rows = math.ceil(len(image_paths)/4)
    fig, ax = plt.subplots(num_rows, 4, figsize=(16, 16))      
    
    if num_rows > 1:
        ax = ax.flatten()
    else:
        ax = [ax]  # Make it iterable for consistency    
    
    imgs = []
    for i, image_path in enumerate(image_paths):
        # Load image
        img = cv2.imread(image_path)
        imgs.append(img)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        gt_boxes = []        
        if label_dir or label_paths:
            # Get corresponding label file
            if label_paths:
                label_path = label_paths[image_path]
            else:
                label_path = os.path.join(label_dir, os.path.basename(image_path).replace('.png', '.txt'))

            if os.path.exists(label_path):
                # Read ground truth labels
                with open(label_path, 'r') as f:
                    gt_labels = f.read()

                gt_labels = clean_newlines(gt_labels)

                # Convert ground truth labels to bounding boxes
                if gt_labels:
                    for label in gt_labels:
                        cls, x, y, w, h = map(float, label.strip().split())
                        x_min = int((x - w / 2) * img.shape[1])
                        y_min = int((y - h / 2) * img.shape[0])
                        x_max = int((x + w / 2) * img.shape[1])
                        y_max = int((y + h / 2) * img.shape[0])
                        gt_boxes.append([x_min, y_min, x_max, y_max, int(cls)])

        # Get prediction results for the current image
        result = results[i]
        pred_boxes = result.boxes.xyxy.cpu().numpy()  # Convert to numpy array
        pred_cls = result.boxes.cls.cpu().numpy()
        pred_conf = result.boxes.conf.cpu().numpy()
        
        ax[i].imshow(img_rgb)
        
        # Plot ground truth boxes
        for box in gt_boxes:
            x_min, y_min, x_max, y_max, cls = box
            rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, fill=False, color='green', linewidth=2)
            ax[i].add_patch(rect)
            ax[i].text(x_min, y_min - 2, f'GT {cls}', fontsize=12, color='green')
        
        # Plot predicted boxes
        for box, cls, conf in zip(pred_boxes, pred_cls, pred_conf):
            x_min, y_min, x_max, y_max = box
            rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, fill=False, color='red', linewidth=2)
            ax[i].add_patch(rect)
            ax[i].text(x_min, y_min - 2, f'Pred {cls:.0f} ({conf:.2f})', fontsize=12, color='red')
        
    # Show the result
    plt.show()
        
    return results, imgs
        
# Example usage:
# model = ...  # Load your YOLO model here
# plot_yolo_predictions_and_ground_truth(
#     image_dir="/kaggle/input/rsna-2024-yolo/yolo_zip/images/val",
#     label_dir="/kaggle/input/rsna-2024-yolo/yolo_zip/labels/val",
#     model=model,
#     pattern="1018005303*"
# )


In [105]:
STUDY_ID = "1008446160_3775545364"
image_dir="/kaggle/working/yolo_set/images/inference/"
pattern=f"{STUDY_ID}*"
os.path.join(image_dir, pattern)


'/kaggle/working/yolo_set/images/inference/1008446160_3775545364*'

In [106]:
study_id_meta[4646740]

{'folder_path': '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/4646740',
 'series_id_files': [3201256954],
 'series_descriptions': ['Axial T2']}

In [107]:
image_paths = sorted(glob.glob(os.path.join(image_dir, pattern)),key=extract_numbers)


In [ ]:
MODEL_DIR = "/kaggle/input/rsna-yolo-v1/pytorch/best_20240716/2/best_20240716.pt" if DATASET_SELECTION=='sagittal' else "/kaggle/input/rsna-yolo-v1/pytorch/best_20240716/4/best_axial_20241106.pt"
# MODEL_DIR='/kaggle/input/rsna-yolo-v1/pytorch/best_20240716/2/best_sagittal_20241105.pt'

if SUBMISSION:
    model = YOLO(MODEL_DIR)
    STUDY_ID = "1777645381_1438513204" # 1018005303 1019430579 105895264 

    results, imgs = plot_yolo_predictions_and_ground_truth(
        image_dir="/kaggle/working/yolo_set/images/inference/",
        label_dir=None,
        model=model,
        pattern=f"{STUDY_ID}*",
        conf=0.25,
        iou=0.8
    )


In [ ]:
SUBMISSION

### 2D Boxes into 3D segments - an algorithm?

Preprocessing:
- Take the highest probability box, for each class per instance.
- [DONE via YOLO?] If there are any boxes with IoU of more than 0.8 take the highest probability, this is regardless of class, as we know the boundaries between different segments are distinct. YOLO might do this per class, but not independent of class.

**At this stage we assume only for Sagittal**, when other MRI types are incorporate, we'll have a list of study_ids for each MRI type (and they'll be indexed by studyid_seriesid.

Approach:
- Take each 2D slice, grab ROI points, and fill in the following dictionary information.
    ```python
    {
    study_id:
        {
        class_0: {instance_no: [], box_coord: [], box_conf: [], left_right: []}
        }
    }

    ```
- Perform calculation to grab 3D segments:
    - If there is a gap of more than 1 instance, then discard outer segment. TODO decide on approach for left/right neuronamial farrowing.
    - Calculate the maximum region across each slice, by adjusting for the positioning of the  x,y coords stacking images, and then taking the maximum area. For example, we essentially want to obtain adjusted, max_x, max_y, min_x, min_y, and that becomes the new bounding box, for the voxel.
    - [POSSIBLE] Grab centre coordinate and discard those where they are > % away from standard distribution of centre coords in dataset

In [109]:
# TODO - add in series_ids
# Set up struture for box metadata
boxes_meta = {}
class_list = [0,1,2,3,4] if DATASET_SELECTION == 'sagittal' else [0,1]
for key, values in study_id_meta.items():
    for series in values['series_id_files']:
        boxes_meta.update({f"{key}_{series}": {i:{'instance_no':[],"box_coord":[],"box_conf":[], "left_right":[]} for i in class_list}})
# slice_result = results[4]

In [110]:
torch.cuda.empty_cache()

In [111]:
import gc
gc.collect()  # Force garbage collection


4727

In [112]:
INFERENCE_BATCH_SIZE = 100
no_batches = (len(files_for_inference)//INFERENCE_BATCH_SIZE)+1
batches_files_for_inference = [files_for_inference[((i*INFERENCE_BATCH_SIZE)):(i+1)*INFERENCE_BATCH_SIZE if (i+1) < no_batches else len(files_for_inference)] for i in range(no_batches)]
assert batches_files_for_inference[no_batches-1][-1] == files_for_inference[-1]

In [113]:
batches_files_for_inference[no_batches-1]

['/kaggle/working/yolo_set/images/inference/991428866_3160509931_26.png',
 '/kaggle/working/yolo_set/images/inference/991428866_3160509931_27.png',
 '/kaggle/working/yolo_set/images/inference/991428866_3160509931_28.png',
 '/kaggle/working/yolo_set/images/inference/991428866_3160509931_29.png',
 '/kaggle/working/yolo_set/images/inference/991428866_3160509931_3.png',
 '/kaggle/working/yolo_set/images/inference/991428866_3160509931_30.png',
 '/kaggle/working/yolo_set/images/inference/991428866_3160509931_31.png',
 '/kaggle/working/yolo_set/images/inference/991428866_3160509931_32.png',
 '/kaggle/working/yolo_set/images/inference/991428866_3160509931_33.png',
 '/kaggle/working/yolo_set/images/inference/991428866_3160509931_34.png',
 '/kaggle/working/yolo_set/images/inference/991428866_3160509931_35.png',
 '/kaggle/working/yolo_set/images/inference/991428866_3160509931_36.png',
 '/kaggle/working/yolo_set/images/inference/991428866_3160509931_37.png',
 '/kaggle/working/yolo_set/images/infer

### Getting around YOLO

YOLO doesn't seem to use GPU memory properly, `stream=True` doesn't decrease GPU memory utlisation enough. So performing one's own batching.

In [114]:
torch.cuda.empty_cache()

In [115]:
for ndx, files in enumerate(batches_files_for_inference):
    
    results = model(files,task='detect',iou=0.8,stream=True, cache=False, batch=1, save=False, verbose=False)  # return a list of Results objects
    
    for slice_result in results:
        if slice_result.boxes:
            img_path = slice_result.path
            study_id, series_id, instance_no = extract_numbers(img_path)
            box = slice_result.boxes
            classes = box.cls.cpu()
            conf = box.conf.cpu()
            coord = box.xywh.cpu()
            id_key = f"{study_id}_{series_id}"
            for i, class_ in enumerate(classes):
                class_ = int(class_)
                boxes_meta[id_key][class_]['instance_no'].append(instance_no)
                boxes_meta[id_key][class_]['box_conf'].append(conf[i])
                boxes_meta[id_key][class_]['box_coord'].append(coord[i])
    
    if ndx%10==0:
        print(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}, Processed {ndx*INFERENCE_BATCH_SIZE} images.")
                

2024-11-12 10:43:49, Processed 0 images.
2024-11-12 10:43:59, Processed 1000 images.
2024-11-12 10:44:09, Processed 2000 images.
2024-11-12 10:44:18, Processed 3000 images.
2024-11-12 10:44:28, Processed 4000 images.
2024-11-12 10:44:38, Processed 5000 images.
2024-11-12 10:44:47, Processed 6000 images.
2024-11-12 10:44:57, Processed 7000 images.
2024-11-12 10:45:06, Processed 8000 images.
2024-11-12 10:45:16, Processed 9000 images.
2024-11-12 10:45:25, Processed 10000 images.
2024-11-12 10:45:35, Processed 11000 images.
2024-11-12 10:45:44, Processed 12000 images.
2024-11-12 10:45:54, Processed 13000 images.
2024-11-12 10:46:04, Processed 14000 images.
2024-11-12 10:46:13, Processed 15000 images.
2024-11-12 10:46:23, Processed 16000 images.
2024-11-12 10:46:32, Processed 17000 images.
2024-11-12 10:46:42, Processed 18000 images.
2024-11-12 10:46:51, Processed 19000 images.
2024-11-12 10:47:00, Processed 20000 images.
2024-11-12 10:47:10, Processed 21000 images.
2024-11-12 10:47:19, Pr

In [116]:
import pickle
with open(f'/kaggle/working/{SET_TYPE}_{DATASET_SELECTION}_boxes_meta.pkl', 'wb') as file:
    pickle.dump(boxes_meta, file)

In [22]:
import pickle
boxes_meta = pickle.load(open(f"/kaggle/input/rsna-2024-train-candidate-list/{SET_TYPE}_{DATASET_SELECTION}_boxes_meta.pkl","rb"))

boxes_meta = pickle.load(open(f"/kaggle/input/rsna-2024-train-candidate-list/train_sagittal_boxes_meta.pkl","rb"))

In [44]:
study_id_meta[4003253]

{'folder_path': '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/4003253',
 'series_id_files': [702807833, 1054713880],
 'series_descriptions': ['Sagittal T2/STIR', 'Sagittal T1']}

In [ ]:
boxes_meta["4003253_1054713880"]

In [ ]:
boxes_meta[list(boxes_meta.keys())[0]], list(boxes_meta.keys())[0]

**Get Image MetaData**

Get dicom image meta data for all Sagittal T1 images, such that we can attribute boxes to L/R estimate.

In [48]:
sagittal_t1_series_ids = [
    f"{key}_{series_id}"
    for key, entry in study_id_meta.items()
    for series_id, description in zip(entry['series_id_files'], entry['series_descriptions'])
    if description == 'Sagittal T1'
]

In [49]:
# TODO DELETE
# Get all image_list, then filter for Sagittal T1
DICOM_DATA_PATH = f"/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/{SET_TYPE}_images/" 
images = glob.glob(os.path.join(DICOM_DATA_PATH,"*/*/*.dcm"), recursive=True)

In [50]:
def rescale_bbox(bbox_xyxy: np.array, original_w_h=()):    
    # Original image size
    original_width = original_w_h[0]
    original_height = original_w_h[1]

    # Resized image size
    resized_width = IMG_RESIZE[0]
    resized_height = IMG_RESIZE[1]

    # Scaling factors
    scale_x = original_width / resized_width
    scale_y = original_height / resized_height

    # Upscale the bounding box coordinates to the original image size
    bbox_original = bbox_xyxy * np.array([scale_x, scale_y, scale_x, scale_y])

    # bbox_original now contains the upscaled coordinates
    x1_upscaled, y1_upscaled, x2_upscaled, y2_upscaled = bbox_original
    
    return (int(x1_upscaled), int(y1_upscaled), int(x2_upscaled), int(y2_upscaled))

In [51]:
from torch import tensor
def get_xyxy(boxes: list, return_int=True) -> tuple:
    """
    Get largest region in xyxy coordinates given a set of multiple slices of xywh coordinates.
    """
    if isinstance(boxes,list):
        boxes = torch.stack(boxes)
    elif isinstance(boxes, torch.Tensor) and len(boxes.shape)==1:
        boxes = boxes.unsqueeze(0)

    x_centers = boxes[:, 0]
    y_centers = boxes[:, 1]
    widths = boxes[:, 2]
    heights = boxes[:, 3]

    x_mins = x_centers - widths / 2
    y_mins = y_centers - heights / 2
    x_maxs = x_centers + widths / 2
    y_maxs = y_centers + heights / 2

    # Compute the desired max and min values
    min_x = torch.min(x_mins).item()
    max_x = torch.max(x_maxs).item()
    min_y = torch.min(y_mins).item()
    max_y = torch.max(y_maxs).item()

    if return_int:
        min_x = int(min_x)
        max_x = int(max_x)
        min_y = int(min_y)
        max_y = int(max_y)
    
    return (min_x, max_x, min_y, max_y)

In [52]:
def get_box_stats(data: dict):
    if data['instance_no']:
        data['roi_xyxy'] = get_xyxy(data['box_coord'])
        max_conf = torch.argmax(torch.tensor(data['box_conf'])).item() 
        data['conf_xyxy'] = get_xyxy(data['box_coord'][max_conf])
        data['conf_instance_no'] = data['instance_no'][max_conf]
    else:
        data['roi_xyxy'] = ()
        data['conf_xyxy'] = ()
        data['conf_instance_no'] = np.NaN
        
        
    

Obtain `new_data`, a bounding box region for each condition and class.

To obtain left/right for neural foraminal narrowing we see use the `instance_number` of dicom meta data. Images are taken right to left, the assumption being images on the right (left) hand side will be below (above) the median/average. This crude method results in 99.4% accuracy on train set.

| left_or_right_class | left_or_right_avg | Count |
|---------------------|-------------------|-------|
| left                | left              | 9790  |
|                     | right             | 70    |
| right               | left              | 41    |
|                     | right             | 9788  |


Reading in descriptive train.csv with all study information.

In [53]:
train_desc = datasets['sagittal'].copy()

train_desc.study_id = train_desc.study_id.astype(str)
train_desc.series_id = train_desc.series_id.astype(str)

In [54]:
x = pd.DataFrame.from_dict(boxes_meta,orient='index')

for col in x.columns:
    x[f'{col}_populated'] = x.apply(lambda x: True if x[col]['instance_no'] else False, axis=1)

# Get study and series id from index
x = x.reset_index()
x.rename(columns={'index': 'study_series'}, inplace=True)
x[['study_id', 'series_id']] = x['study_series'].str.split('_', expand=True)


x.dtypes, train_desc.dtypes

x.head()

,study_series,0,1,2,3,4,0_populated,1_populated,2_populated,3_populated,4_populated,study_id,series_id
0,4003253_702807833,"{'instance_no': [11, 6], 'box_coord': [[tensor...","{'instance_no': [12, 12, 5, 6], 'box_coord': [...","{'instance_no': [12, 5], 'box_coord': [[tensor...","{'instance_no': [11, 4], 'box_coord': [[tensor...","{'instance_no': [11, 3, 4, 4], 'box_coord': [[...",True,True,True,True,True,4003253,702807833
1,4003253_1054713880,"{'instance_no': [11, 6], 'box_coord': [[tensor...","{'instance_no': [11, 12, 5, 6], 'box_coord': [...","{'instance_no': [11, 12, 5], 'box_coord': [[te...","{'instance_no': [11, 12, 4], 'box_coord': [[te...","{'instance_no': [11, 12, 3, 4], 'box_coord': [...",True,True,True,True,True,4003253,1054713880
2,4646740_3486248476,"{'instance_no': [16, 17, 6, 7, 8], 'box_coord'...","{'instance_no': [15, 16, 5, 6], 'box_coord': [...","{'instance_no': [15, 4, 5], 'box_coord': [[ten...","{'instance_no': [16, 17, 3, 4, 5], 'box_coord'...","{'instance_no': [5, 6, 7], 'box_coord': [[tens...",True,True,True,True,True,4646740,3486248476
3,4646740_3666319702,"{'instance_no': [16, 5], 'box_coord': [[tensor...","{'instance_no': [], 'box_coord': [], 'box_conf...","{'instance_no': [], 'box_coord': [], 'box_conf...","{'instance_no': [17], 'box_coord': [[tensor(25...","{'instance_no': [], 'box_coord': [], 'box_conf...",True,False,False,True,False,4646740,3666319702
4,7143189_132939515,"{'instance_no': [], 'box_coord': [], 'box_conf...","{'instance_no': [], 'box_coord': [], 'box_conf...","{'instance_no': [], 'box_coord': [], 'box_conf...","{'instance_no': [], 'box_coord': [], 'box_conf...","{'instance_no': [], 'box_coord': [], 'box_conf...",False,False,False,False,False,7143189,132939515


In [55]:
train_pred = train_desc.merge(right=x, how='left',on=['study_id','series_id'])

In [61]:
assert any(train_pred.study_series.isna())==False

In [62]:
pop_cols = ["0_populated","1_populated","2_populated","3_populated","4_populated"]
for pop_col in pop_cols:
    print(train_pred.groupby(['series_description', pop_col]).size())

series_description  0_populated
Sagittal T1         False             2
                    True           1978
Sagittal T2/STIR    False           387
                    True           1587
dtype: int64
series_description  1_populated
Sagittal T1         False             8
                    True           1972
Sagittal T2/STIR    False           505
                    True           1469
dtype: int64
series_description  2_populated
Sagittal T1         False             7
                    True           1973
Sagittal T2/STIR    False           521
                    True           1453
dtype: int64
series_description  3_populated
Sagittal T1         False             8
                    True           1972
Sagittal T2/STIR    False           554
                    True           1420
dtype: int64
series_description  4_populated
Sagittal T1         False             3
                    True           1977
Sagittal T2/STIR    False           368
                    True    

In [ ]:
x.dtypes

In [ ]:
pop_cols = ['0_populated', '1_populated', '2_populated', '3_populated', '4_populated']

Analysing the predictions from YOLO

In [ ]:
for col in pop_cols:
    print(train_pred.groupby(col).size())

In [ ]:
# TODO two approaches (1) use simple look up against metadata_df, for the instance number on whether left/right, (2) calculate the averages within the box (which 
# heavily relies on both right/left predictions existing)

# Approach (2) here
new_data = {}
for study_series, items in boxes_meta.items():
    preds = items
#     for preds in items.values(): # Not at series level yet

    # Separate classes into left/right based on position of instance number
    for level in class_list:
        
        level_class = LUMBAR_INT_MAPPING[level]
                
        if study_series not in sagittal_t1_series_ids:
            inner_value = preds[level]
            get_box_stats(inner_value)
            
            new_data[f'{study_series}_spinal_canal_stenosis_{level_class}'] = inner_value            
            
            
        else:
        
            if level in preds.keys() and preds[level]['instance_no']:
                instance_nos = preds[level]['instance_no']
                try:
                    avg_in = np.mean(instance_nos)
                except Exception:
                    print(instance_nos)
                preds[level]['left_right'] = ["left" if i > avg_in else "right" for i in instance_nos]

            inner_value = preds[level]
            left_data = {'instance_no': [], 'box_coord': [], 'box_conf': [], 'roi_xyxy': ()}
            right_data = {'instance_no': [], 'box_coord': [], 'box_conf': [], 'roi_xyxy': ()}

            for i, lr in enumerate(inner_value['left_right']):
                if lr == 'left':
                    left_data['instance_no'].append(inner_value['instance_no'][i])
                    left_data['box_coord'].append(inner_value['box_coord'][i])
                    left_data['box_conf'].append(inner_value['box_conf'][i])
                elif lr == 'right':
                    right_data['instance_no'].append(inner_value['instance_no'][i])
                    right_data['box_coord'].append(inner_value['box_coord'][i])
                    right_data['box_conf'].append(inner_value['box_conf'][i])

            if right_data['box_coord']:
                get_box_stats(right_data)

            if left_data['box_coord']:
                get_box_stats(left_data)

            if left_data['instance_no']:
                new_data[f'{study_series}_left_neural_foraminal_narrowing_{level_class}'] = left_data
            if right_data['instance_no']:
                new_data[f'{study_series}_right_neural_foraminal_narrowing_{level_class}'] = right_data

In [ ]:
for key, value in new_data.items():
    study_id, series_id = tuple(key.split("_")[0:2])
    instance = value['conf_instance_no']
    if np.isnan(instance):
        continue
    img_path = f"{DATA_PATH}/{SET_TYPE}_images/{study_id}/{series_id}/{instance}.dcm"
    temp = get_image(img_path)
    
    set_type = study_id_meta[int(study_id)]['series_descriptions'][
                    study_id_meta[int(study_id)]['series_id_files'].index(int(series_id))
    ]
    
    # TODO update image path and all that other good stuff here
    value['img_path'] = img_path
    value['study_id'] = study_id
    value['series_id'] = series_id
    value['set_type'] = set_type

    value['roi_xyxy'] = rescale_bbox(np.array(value['roi_xyxy']), (temp.Rows, temp.Columns))
    value['conf_xyxy'] = rescale_bbox(np.array(value['conf_xyxy']), (temp.Rows, temp.Columns))
    

In [ ]:
import pickle

In [ ]:
with open(f'/kaggle/working/{SET_TYPE}_sagittal_boxes.pkl', 'wb') as file:
    pickle.dump(new_data, file)